# Excess mortality in Russia due to the war in Ukraine

In [31]:
%matplotlib inline

import numpy as np
import pylab as plt
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px



from sklearn.linear_model import LinearRegression

plt.style.use("mpl_style.txt")

In [14]:
df = pd.read_csv("C:/Users/aagal/projects/excess-mortality-war/deaths-by-age-gender-region-year/deaths-by-age-gender-region-year-1990-2023.csv.gz")

df

,Region,Year,Age,Gender,Deaths
0,Алтайский край,1990,0-4 лет,f,264
1,Алтайский край,1990,10-14 лет,f,28
2,Алтайский край,1990,15-19 лет,f,63
3,Алтайский край,1990,20-24 лет,f,67
4,Алтайский край,1990,25-29 лет,f,94
...,...,...,...,...,...
116691,Ярославская область,2023,75-79 лет,m,681
116692,Ярославская область,2023,80-84 лет,m,727
116693,Ярославская область,2023,85 и более,m,733
116694,Ярославская область,2023,Неизвестно,m,5


In [15]:
def get_excess(region):
    LinReg = LinearRegression()
    
    agegroups = ['15-19 лет', '20-24 лет', '25-29 лет',
                 '30-34 лет', '35-39 лет', '40-44 лет', 
                 '45-49 лет', '50-54 лет',  '55-59 лет']
    
    excess_male = [0, 0]          # For penultimate and last year
    total_baseline = [0, 0]
    excess_male_var = [0, 0]

    for age in agegroups:
        male = df[
            (df.Region == region) & 
            (df.Age == age) & 
            (df.Gender == 'm') & 
            (df.Year >= 2015)
        ][['Year', 'Deaths']].values
        
        female = df[
            (df.Region == region) & 
            (df.Age == age) & 
            (df.Gender == 'f') & 
            (df.Year >= 2015)
        ][['Year', 'Deaths']].values
        
        if len(male) < 7 or len(female) < 7:
            continue  # skip incomplete data
        
        ratio = male[:,1] / (1 + female[:,1])
        
        LinReg.fit(male[:5, 0].reshape(-1,1), ratio[:5].reshape(-1,1))

        for i, offset in enumerate([-2, -1]):  # second-last and last year
            ratio_hat = LinReg.predict(male[offset, 0].reshape(-1,1))[0][0]
            ratio_hat = max(ratio_hat, 1)
            yhat = (1 + female[offset, 1]) * ratio_hat
            excess_male[i] += male[offset, 1] - yhat
            total_baseline[i] += yhat

            # Compute predictive variance
            X = np.concatenate((male[:5,0].reshape(-1,1), np.ones((5,1))), axis=1)
            y = ratio[:5].reshape(-1,1)
            beta = np.linalg.pinv(X.T @ X) @ X.T @ y
            sigma2 = np.sum((y - X @ beta)**2) / (len(y) - 2)
            S = np.linalg.pinv(X.T @ X)
            x_new = np.array([[male[offset, 0], 1]])
            pred_var = sigma2 * (x_new @ S @ x_new.T + 1)
            excess_male_var[i] += pred_var[0][0] * (1 + female[offset,1])**2

    excess_pratio = [excess_male[i] / total_baseline[i] if total_baseline[i] > 0 else 0 for i in range(2)]
    z = [excess_male[i] / np.sqrt(excess_male_var[i]) if excess_male_var[i] > 0 else 0 for i in range(2)]
    
    return excess_male[0], excess_pratio[0], np.sqrt(excess_male_var[0]), z[0], \
           excess_male[1], excess_pratio[1], np.sqrt(excess_male_var[1]), z[1]


In [16]:
regions = df.Region.unique()

excess_regional = np.zeros((len(regions), 8)) 
for i, region in enumerate(regions):
    print('.', end='')
    excess_regional[i] = get_excess(region)


........................................................................................

In [17]:
# Skip regions with AOs if needed
good = ~np.isin(regions, ['Архангельская область', 'Тюменская область'])

df_both_years = pd.DataFrame({
    'Region': regions[good],
    
    # 2021 (penultimate year)
    'Excess 2022': excess_regional[:, 0][good],
    'Excess SE 2022': excess_regional[:, 2][good],
    'Z-score 2022': excess_regional[:, 3][good],
    'P-score 2022': excess_regional[:, 1][good],
    
    # 2022 (last year)
    'Excess 2023': excess_regional[:, 4][good],
    'Excess SE 2023': excess_regional[:, 6][good],
    'Z-score 2023': excess_regional[:, 7][good],
    'P-score 2023': excess_regional[:, 5][good],
})

# Export to CSV
df_both_years.to_csv('regional-excess.csv', float_format='%.2f', index=False)


In [18]:
df_both_years = df_both_years[df_both_years['Region'] != 'Российская Федерация']


# Normalising by population

In [19]:
male_pop = pd.read_excel("C:/Users/aagal/projects/excess-mortality-war/male_pop.xls")
male_pop.columns.values[0:2] = ['age', 'Region']
male_pop['Region'] = male_pop['Region'].str.strip()
agegroups = ['15-19 лет', '20-24 лет', '25-29 лет',
             '30-34 лет', '35-39 лет', '40-44 лет', 
             '45-49 лет', '50-54 лет',  '55-59 лет']

male_pop_long = male_pop.melt(
    id_vars=['age', 'Region'],
    var_name='year',
    value_name='male_population'
)

male_pop_long['year'] = pd.to_numeric(male_pop_long['year'], errors='coerce')
male_pop_long = male_pop_long[male_pop_long['age'].isin(agegroups)]
male_pop_long = male_pop_long.dropna(subset=['male_population'])
male_pop_long = male_pop_long[male_pop_long['year'].isin([2022, 2023])]

male_pop1549 = male_pop_long.groupby(['Region', 'year'], as_index=False)['male_population'].sum()


In [20]:
region_name_mapping = {
    'Архангельская область (кроме Ненецкого автономного округа)': 'Архангельская область без АО',
    'Ненецкий автономный округ (Архангельская область)': 'Ненецкий АО',
    'Кемеровская область - Кузбасс': 'Кемеровская область',
    'Город Москва столица Российской Федерации город федерального значения': 'Москва',
    'Город Санкт-Петербург город федерального значения': 'Санкт-Петербург',
    'Город федерального значения Севастополь': 'Севастополь',
    'Еврейская автономная область': 'Еврейская АО',
    'Кабардино-Балкарская Республика': 'Кабардино-Балкария',
    'Карачаево-Черкесская Республика': 'Карачаево-Черкесия',
    'Республика Адыгея (Адыгея)': 'Республика Адыгея',
    'Республика Саха (Якутия)': 'Якутия',
    'Республика Северная Осетия-Алания': 'Северная Осетия',
    'Республика Татарстан (Татарстан)': 'Республика Татарстан',
    'Чувашская Республика - Чувашия': 'Чувашская Республика',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)': 'Тюменская область без АО',
    'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский АО',
    'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Hенецкий АО',
    'Чукотский автономный округ': 'Чукотский АО',
}

male_pop1549['Region'] = male_pop1549['Region'].replace(region_name_mapping)



In [21]:
set(df_both_years.Region.unique()).symmetric_difference(set(male_pop1549.Region.unique()))

{'Архангельская область',
 'Дальневосточный федеральный округ (с 03.11.2018)',
 'Приволжский федеральный округ',
 'Российская Федерация',
 'Северо-Западный федеральный округ',
 'Северо-Кавказский федеральный округ',
 'Сибирский федеральный округ (с 03.11.2018)',
 'Тюменская область',
 'Уральский федеральный округ',
 'Центральный федеральный округ',
 'Южный федеральный округ (с 29.07.2016)'}

In [22]:
pop_pivot = male_pop1549.pivot(index='Region', columns='year', values='male_population').reset_index()
pop_pivot.columns.name = None  # Remove the name of the columns axis
pop_pivot = pop_pivot.rename(columns={
    2022: 'Male pop2022',
    2023: 'Male pop2023'
})

df_merged = df_both_years.merge(pop_pivot, on='Region', how='left')
df_merged

,Region,Excess 2022,Excess SE 2022,Z-score 2022,P-score 2022,Excess 2023,Excess SE 2023,Z-score 2023,P-score 2023,Male pop2022,Male pop2023
0,Алтайский край,128.246418,244.472443,0.524584,0.024355,691.156780,261.281037,2.645262,0.147973,584819.0,576369.0
1,Амурская область,504.416315,148.047794,3.407118,0.278132,589.317060,182.887125,3.222299,0.308756,228799.0,226425.0
2,Архангельская область без АО,-96.315932,292.762215,-0.328990,-0.036983,263.246517,273.871610,0.961204,0.110666,276243.0,272674.0
3,Астраханская область,297.114738,150.377799,1.975789,0.173459,493.048096,170.111802,2.898377,0.299916,273148.0,270504.0
4,Белгородская область,355.700355,184.568346,1.927201,0.129426,640.925314,190.449703,3.365326,0.251041,442249.0,433216.0
...,...,...,...,...,...,...,...,...,...,...,...
80,Чувашская Республика,-248.799524,270.080898,-0.921204,-0.080263,220.980515,267.046248,0.827499,0.080414,334430.0,328223.0
81,Чукотский АО,1.974285,61.091478,0.032317,0.010228,17.702778,58.188232,0.304233,0.092541,16398.0,16096.0
82,Якутия,60.135241,123.586523,0.486584,0.030236,326.914104,126.996129,2.574205,0.182523,309111.0,307385.0
83,Ямало-Hенецкий АО,-57.060639,153.619495,-0.371441,-0.053475,48.251756,162.768705,0.296444,0.048801,167641.0,165779.0


In [23]:
df_merged['Excess per100k (2022)'] = df_merged['Excess 2022'] / df_merged['Male pop2022'] * 100_000
df_merged['Excess per100k (2023)'] = df_merged['Excess 2023'] / df_merged['Male pop2023'] * 100_000


In [ ]:

fig = px.scatter(
    df_merged,
    x='Excess per100k (2022)',
    y='Excess per100k (2023)',
    hover_name='Region',
    title="Excess Male Mortality: 2022 vs 2023",
    labels={"Excess 2022": "Excess in 2022", "Excess 2023": "Excess in 2023"},
    width=800, height=700
)



fig.show()


# Crowdsourced data on mortality 2022-2023 (BBC and Mediazona)


In [24]:
total = pd.read_csv("C:/Users/aagal/projects/excess-mortality-war/total2223.csv")
prisoners  = pd.read_csv("C:/Users/aagal/projects/excess-mortality-war/prisoners2223.csv")
pmc = pd.read_csv("C:/Users/aagal/projects/excess-mortality-war/pmc2223.csv")
volunteers = pd.read_csv("C:/Users/aagal/projects/excess-mortality-war/volunteers2223.csv")
drafted = pd.read_csv("C:/Users/aagal/projects/excess-mortality-war/drafted2223.csv")

In [25]:
drafted = drafted.rename(columns={'value': 'Drafted'})
volunteers = volunteers.rename(columns={'value': 'Volunteers'})
prisoners = prisoners.rename(columns={'value': 'Prisoners'})
pmc = pmc.rename(columns={'value': 'PMC'})
mz_data = total.rename(columns={'value': 'Total'})  

mz_data = mz_data.merge(drafted, on='region', how='left')
mz_data = mz_data.merge(volunteers, on='region', how='left')
mz_data = mz_data.merge(prisoners, on='region', how='left')
mz_data = mz_data.merge(pmc, on='region', how='left')

mz_data = mz_data.rename(columns={'region':"Region"})

mz_data


,Region,Total,Drafted,Volunteers,Prisoners,PMC
0,###,570,14.0,35,271,23.0
1,Иностранцы,370,NaN,117,138,30.0
2,Санкт-Петербург,1022,127.0,249,125,50.0
3,Москва,1325,191.0,324,134,54.0
4,ЛНР,92,3.0,21,8,10.0
...,...,...,...,...,...,...
84,Магаданская область,169,41.0,22,41,6.0
85,Хабаровский край,658,66.0,136,93,13.0
86,Приморский край,1881,194.0,336,274,28.0
87,Камчатский край,427,43.0,101,61,5.0


In [26]:
region_name_mapping = {
    'Архангельская область': 'Архангельская область без АО',
    'Ненецкий автономный округ': 'Ненецкий АО',
    'Еврейская автномная область': 'Еврейская АО',
    'Кемеровская область - Кузбасс': 'Кемеровская область',
    'Еврейская автономная область': 'Еврейская АО',
    'Кабардино-Балкарская Республика': 'Кабардино-Балкария',
    'Республика Карачаево-Черкесия': 'Карачаево-Черкесия',
    'Республика Адыгея (Адыгея)': 'Республика Адыгея',
    'Республика Саха (Якутия)': 'Якутия',
    'Республика Северная Осетия-Алания': 'Северная Осетия',
    'Тюменская область': 'Тюменская область без АО',
    'Ханты-Мансийский автономный округ - Югра': 'Ханты-Мансийский АО',
    'Ямало-Ненецкий автономный округ': 'Ямало-Hенецкий АО',
    'Чукотский автономный округ': 'Чукотский АО',
}

mz_data['Region'] = mz_data['Region'].replace(region_name_mapping)



In [27]:
df_merged = df_merged.merge(mz_data, on = "Region", how = 'left')
df_merged

,Region,Excess 2022,Excess SE 2022,Z-score 2022,P-score 2022,Excess 2023,Excess SE 2023,Z-score 2023,P-score 2023,Male pop2022,Male pop2023,Excess per100k (2022),Excess per100k (2023),Total,Drafted,Volunteers,Prisoners,PMC
0,Алтайский край,128.246418,244.472443,0.524584,0.024355,691.156780,261.281037,2.645262,0.147973,584819.0,576369.0,21.929250,119.915675,2051,235.0,582,288,52.0
1,Амурская область,504.416315,148.047794,3.407118,0.278132,589.317060,182.887125,3.222299,0.308756,228799.0,226425.0,220.462640,260.270315,507,47.0,91,85,4.0
2,Архангельская область без АО,-96.315932,292.762215,-0.328990,-0.036983,263.246517,273.871610,0.961204,0.110666,276243.0,272674.0,-34.866379,96.542581,1237,175.0,356,255,36.0
3,Астраханская область,297.114738,150.377799,1.975789,0.173459,493.048096,170.111802,2.898377,0.299916,273148.0,270504.0,108.774268,182.270168,1043,95.0,259,232,15.0
4,Белгородская область,355.700355,184.568346,1.927201,0.129426,640.925314,190.449703,3.365326,0.251041,442249.0,433216.0,80.429883,147.945901,1169,171.0,276,114,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Чувашская Республика,-248.799524,270.080898,-0.921204,-0.080263,220.980515,267.046248,0.827499,0.080414,334430.0,328223.0,-74.395097,67.326335,951,87.0,277,109,18.0
81,Чукотский АО,1.974285,61.091478,0.032317,0.010228,17.702778,58.188232,0.304233,0.092541,16398.0,16096.0,12.039790,109.982468,114,16.0,63,14,NaN
82,Якутия,60.135241,123.586523,0.486584,0.030236,326.914104,126.996129,2.574205,0.182523,309111.0,307385.0,19.454255,106.353304,1203,238.0,329,178,20.0
83,Ямало-Hенецкий АО,-57.060639,153.619495,-0.371441,-0.053475,48.251756,162.768705,0.296444,0.048801,167641.0,165779.0,-34.037401,29.106072,499,50.0,161,71,7.0


In [28]:
##per capita

df_merged["Drafted"] = df_merged["Drafted"]/df_merged["Male pop2022"]*100000
df_merged["Total"] = df_merged["Total"]/df_merged["Male pop2022"]*100000
df_merged["Volunteers"] = df_merged["Volunteers"]/df_merged["Male pop2022"]*100000
df_merged["Prisoners"] = df_merged["Prisoners"]/df_merged["Male pop2022"]*100000
df_merged["PMC"] = df_merged["PMC"]/df_merged["Male pop2022"]*100000

In [29]:
df_merged["Excess total"] = df_merged["Excess per100k (2022)"] + df_merged["Excess per100k (2023)"]

In [ ]:

fig = px.scatter(
    df_merged,
    x='Excess total',
    y='Total',
    hover_name='Region',
    title="War Fatalities per 100k of Male population aged 18-59",
    labels={"Excess total": "Excess in mortality", "Total": "Crowdsourced data"},
    width=800, height=700
)



fig.show()


In [52]:

top10_x = df_merged.nlargest(10, 'Excess total')[['Region', 'Excess total']]
bottom10_x = df_merged.nsmallest(10, 'Excess total')[['Region', 'Excess total']]
top10_y = df_merged.nlargest(10, 'Total')[['Region', 'Total']]
bottom10_y = df_merged.nsmallest(10, 'Total')[['Region', 'Total']]

top10_x.reset_index(drop=True, inplace=True)
bottom10_x.reset_index(drop=True, inplace=True)
top10_y.reset_index(drop=True, inplace=True)
bottom10_y.reset_index(drop=True, inplace=True)

# Combine into one DataFrame
combined = pd.concat([
    top10_x.rename(columns={'Region': 'Top Regions', 'Excess total': 'Excessive Male Mortality'}),
    top10_y.rename(columns={'Region': 'Top Regions', 'Total': 'Crowdsourced Data'}),
    bottom10_x.rename(columns={'Region': 'Bottom Regions', 'Excess total': 'Excessive Male Mortality'}),
    bottom10_y.rename(columns={'Region': 'Bottom Regions', 'Total': 'Crowdosourced Data'}),
], axis=1)

combined


,Top Regions,Excessive Male Mortality,Top Regions,Crowdsourced Data,Bottom Regions,Excessive Male Mortality,Bottom Regions,Crowdosourced Data
0,Амурская область,480.732954,Республика Тыва,1188.992337,Республика Калмыкия,-299.108649,Москва,35.090935
1,Удмуртская Республика,409.170600,Республика Бурятия,908.960522,Республика Ингушетия,-227.349315,Санкт-Петербург,61.656782
2,Севастополь,407.456493,Республика Алтай,890.217974,Камчатский край,-136.356851,Чеченская Республика,74.858351
3,Курская область,403.774712,Чукотский АО,695.206733,Республика Алтай,-100.651040,Республика Ингушетия,85.517073
4,Ненецкий АО,400.312561,Сахалинская область,655.710201,Калининградская область,-40.476499,Кабардино-Балкария,101.699571
5,Ульяновская область,378.334783,Забайкальский край,596.446147,Карачаево-Черкесия,-26.793890,Московская область,122.114216
6,Приморский край,371.803681,Кировская область,574.928495,Мурманская область,-24.908136,Тульская область,140.837797
7,Сахалинская область,328.312506,Ненецкий АО,554.850983,Вологодская область,-13.776762,Республика Дагестан,149.403490
8,Костромская область,327.447129,Республика Карелия,514.408192,Рязанская область,-7.280998,Карачаево-Черкесия,151.858617
9,Республика Мордовия,313.729046,Республика Хакасия,513.122246,Чувашская Республика,-7.068763,Севастополь,154.926484


In [102]:
nup = pd.read_excel("C:/Users/aagal/projects/excess-mortality-war/data.xls", header=[0, 1])
nup = nup.drop(columns = nup.columns[0])
names = pd.read_excel("C:/Users/aagal/projects/excess-mortality-war/data.xls").iloc[1:,0]
nup.index = names
nup.index.name = "Region"
nup.columns = pd.MultiIndex.from_tuples(nup.columns)

nup

2006                                        \
                             январь февраль   март  апрель    май    июнь   
Region                                                                      
Белгородская область          686.0   751.0  639.0   809.0  427.0  1240.0   
Брянская область              471.0   625.0  554.0   665.0  239.0  1062.0   
Владимирская область          473.0   623.0  601.0   779.0  292.0  1098.0   
Воронежская область           945.0  1042.0  785.0  1254.0  623.0  1544.0   
Ивановская область            347.0   438.0  447.0   529.0  206.0   786.0   
...                             ...     ...    ...     ...    ...     ...   
Амурская область              317.0   442.0  613.0   604.0  218.0   688.0   
Магаданская область            89.0    99.0  126.0   127.0   47.0   165.0   
Сахалинская область           248.0   282.0  333.0   364.0  185.0   422.0   
Еврейская автономная область   78.0    67.0  119.0   114.0   50.0   171.0   
Чукотский автономный округ     11.0    42.0   50.0    41.0   36.0    38.0   

                                                               ...   2024  \
                                июль  август сентябрь октябрь  ... апрель   
Region                                                         ...          
Белгородская область          1581.0  1787.0   1787.0  1140.0  ...    560   
Брянская область              1349.0  1596.0   1427.0   765.0  ...    370   
Владимирская область          1397.0  1689.0   1707.0   884.0  ...    436   
Воронежская область           2121.0  2320.0   2852.0  1434.0  ...    815   
Ивановская область            1062.0  1299.0   1198.0   706.0  ...    321   
...                              ...     ...      ...     ...  ...    ...   
Амурская область               756.0   982.0    808.0   580.0  ...    502   
Магаданская область            162.0   123.0    163.0   151.0  ...     77   
Сахалинская область            437.0   541.0    547.0   456.0  ...    262   
Еврейская автономная область   128.0   199.0    169.0   169.0  ...    100   
Чукотский автономный округ      65.0    31.0     37.0    31.0  ...     28   

                                                                              \
                              май  июнь  июль август сентябрь октябрь ноябрь   
Region                                                                         
Белгородская область          522  1019  1011   1328      889     697    588   
Брянская область              264   754   796    966      597     438    378   
Владимирская область          378   870  1026   1161      800     590    496   
Воронежская область           935  1687  1837   2260     1537    1231    889   
Ивановская область            266   554   625    789      507     400    357   
...                           ...   ...   ...    ...      ...     ...    ...   
Амурская область              348   559   675    720      543     487    400   
Магаданская область            48    89    83     84       79      68     61   
Сахалинская область           210   334   370    464      409     333    284   
Еврейская автономная область   67   122   105    124       55     106     90   
Чукотский автономный округ     22    25    20     29       28      29     31   

                                       2025  
                             декабрь январь  
Region                                       
Белгородская область             482    558  
Брянская область                 370    352  
Владимирская область             426    385  
Воронежская область              723    870  
Ивановская область               303    272  
...                              ...    ...  
Амурская область                 428    304  
Магаданская область               70     57  
Сахалинская область              266    202  
Еврейская автономная область      67     70  
Чукотский автономный округ        26     28  

[87 rows x 229 columns]

In [103]:
nup_long = nup.stack(level=[0, 1]).reset_index()
nup_long.columns = ['Region', 'Year', 'Month', 'Marriages']
month_map = {
    'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4, 'май': 5, 'июнь': 6,
    'июль': 7, 'август': 8, 'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
}
nup_long['Month'] = nup_long['Month'].map(month_map)
nup_long['Year'] = nup_long['Year'].astype(int)

nup_long

C:\Users\aagal\AppData\Local\Temp\ipykernel_12272\2606162799.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



,Region,Year,Month,Marriages
0,Белгородская область,2006,1,686.0
1,Белгородская область,2006,2,751.0
2,Белгородская область,2006,3,639.0
3,Белгородская область,2006,4,809.0
4,Белгородская область,2006,5,427.0
...,...,...,...,...
19510,Чукотский автономный округ,2024,9,28.0
19511,Чукотский автономный округ,2024,10,29.0
19512,Чукотский автономный округ,2024,11,31.0
19513,Чукотский автономный округ,2024,12,26.0


In [104]:
baseline = nup_long[nup_long['Year'].between(2015, 2019) & nup_long["Month"].between(9,11)]
expected = (
    baseline
    .groupby(['Region', 'Month'])['Marriages']
    .mean()
    .reset_index()
    .rename(columns={'Marriages': 'Expected'})
)

expected = expected.merge(nup_long[nup_long["Year"] == 2022], on = ['Region', 'Month'], how = 'left')
expected["Excess"] = expected["Marriages"] - expected["Expected"]
expected

,Region,Month,Expected,Year,Marriages,Excess
0,Архангельская область (кроме Ненец...,9,738.2,2022,785.0,46.8
1,Архангельская область (кроме Ненец...,10,518.0,2022,860.0,342.0
2,Архангельская область (кроме Ненец...,11,482.6,2022,778.0,295.4
3,Ненецкий автономный округ (Арханге...,9,23.6,2022,20.0,-3.6
4,Ненецкий автономный округ (Арханге...,10,17.6,2022,43.0,25.4
...,...,...,...,...,...,...
256,Чукотский автономный округ,10,29.8,2022,77.0,47.2
257,Чукотский автономный округ,11,35.0,2022,48.0,13.0
258,Ярославская область,9,973.2,2022,952.0,-21.2
259,Ярославская область,10,586.2,2022,1020.0,433.8


In [105]:
mar_ex = (
    expected
    .groupby(["Region"])["Excess"]
    .sum()
)
mar_ex.name = "Excessive Marriages"
mar_ex = mar_ex.reset_index()
mar_ex


,Region,Excessive Marriages
0,Архангельская область (кроме Ненец...,684.2
1,Ненецкий автономный округ (Арханге...,22.4
2,Тюменская область (кроме Ханты-Ман...,1886.4
3,Ханты-Мансийский автономный округ ...,868.8
4,Ямало-Ненецкий автономный округ (Т...,386.2
...,...,...
82,Челябинская область,2541.8
83,Чеченская Республика,-885.6
84,Чувашская Республика - Чувашия,185.6
85,Чукотский автономный округ,69.4


# Excessive Nuptiality vs Casualties 

In [106]:
region_name_mapping = {
    'Архангельская область (кроме Ненецкого автономного округа)': 'Архангельская область без АО',
    'Ненецкий автономный округ (Архангельская область)': 'Ненецкий АО',
    'Кемеровская область - Кузбасс': 'Кемеровская область',
    'Город Москва столица Российской Федерации город федерального значения': 'Москва',
    'Город Санкт-Петербург город федерального значения': 'Санкт-Петербург',
    'Город федерального значения Севастополь': 'Севастополь',
    'Еврейская автономная область': 'Еврейская АО',
    'Кабардино-Балкарская Республика': 'Кабардино-Балкария',
    'Карачаево-Черкесская Республика': 'Карачаево-Черкесия',
    'Республика Адыгея (Адыгея)': 'Республика Адыгея',
    'Республика Саха (Якутия)': 'Якутия',
    'Республика Северная Осетия-Алания': 'Северная Осетия',
    'Республика Татарстан (Татарстан)': 'Республика Татарстан',
    'Чувашская Республика - Чувашия': 'Чувашская Республика',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)': 'Тюменская область без АО',
    'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский АО',
    'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Hенецкий АО',
    'Чукотский автономный округ': 'Чукотский АО',
}

mar_ex['Region'] = mar_ex['Region'].str.strip().replace(region_name_mapping)



In [107]:
set(mar_ex.Region.unique()).symmetric_difference(set(df_merged.Region.unique()))

{'Архангельская область', 'Тюменская область'}

In [108]:
df_full = df_merged.merge(mar_ex, on = "Region", how = "left")
df_full["Excessive Marriages per100k"] = df_full["Excessive Marriages"]/df_full["Male pop2022"]*100000
df_full

,Region,Excess 2022,Excess SE 2022,Z-score 2022,P-score 2022,Excess 2023,Excess SE 2023,Z-score 2023,P-score 2023,Male pop2022,...,Excess per100k (2022),Excess per100k (2023),Total,Drafted,Volunteers,Prisoners,PMC,Excess total,Excessive Marriages,Excessive Marriages per100k
0,Алтайский край,128.246418,244.472443,0.524584,0.024355,691.156780,261.281037,2.645262,0.147973,584819.0,...,21.929250,119.915675,350.706800,40.183373,99.517971,49.246006,8.891640,141.844925,2007.6,343.285700
1,Амурская область,504.416315,148.047794,3.407118,0.278132,589.317060,182.887125,3.222299,0.308756,228799.0,...,220.462640,260.270315,221.591878,20.542048,39.772901,37.150512,1.748259,480.732954,1890.4,826.227387
2,Архангельская область без АО,-96.315932,292.762215,-0.328990,-0.036983,263.246517,273.871610,0.961204,0.110666,276243.0,...,-34.866379,96.542581,447.794152,63.350022,128.872044,92.310031,13.032004,61.676202,684.2,247.680484
3,Астраханская область,297.114738,150.377799,1.975789,0.173459,493.048096,170.111802,2.898377,0.299916,273148.0,...,108.774268,182.270168,381.844275,34.779680,94.820390,84.935639,5.491528,291.044437,172.8,63.262407
4,Белгородская область,355.700355,184.568346,1.927201,0.129426,640.925314,190.449703,3.365326,0.251041,442249.0,...,80.429883,147.945901,264.330728,38.666000,62.408281,25.777334,13.793135,228.375784,1182.4,267.360695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Чувашская Республика,-248.799524,270.080898,-0.921204,-0.080263,220.980515,267.046248,0.827499,0.080414,334430.0,...,-74.395097,67.326335,284.364441,26.014413,82.827498,32.592770,5.382292,-7.068763,185.6,55.497414
81,Чукотский АО,1.974285,61.091478,0.032317,0.010228,17.702778,58.188232,0.304233,0.092541,16398.0,...,12.039790,109.982468,695.206733,97.572875,384.193194,85.376265,NaN,122.022258,69.4,423.222344
82,Якутия,60.135241,123.586523,0.486584,0.030236,326.914104,126.996129,2.574205,0.182523,309111.0,...,19.454255,106.353304,389.180586,76.994995,106.434258,57.584492,6.470168,125.807559,850.0,274.982126
83,Ямало-Hенецкий АО,-57.060639,153.619495,-0.371441,-0.053475,48.251756,162.768705,0.296444,0.048801,167641.0,...,-34.037401,29.106072,297.659880,29.825639,96.038559,42.352408,4.175590,-4.931328,386.2,230.373238


# War mortality (Crowdsourced data) and Excessive Nuptiality 

In [ ]:
top10_x = df_full.nlargest(10, 'Excessive Marriages per100k')[['Region', 'Excessive Marriages per100k']]
top10_y = df_full.nlargest(10, 'Total')[['Region', 'Total']]
bottom10_x = df_full.nsmallest(10, 'Excessive Marriages per100k')[['Region', 'Excessive Marriages per100k']]
bottom10_y = df_full.nsmallest(10, 'Total')[['Region', 'Total']]

top10_x.reset_index(drop=True, inplace=True)
top10_y.reset_index(drop=True, inplace=True)
bottom10_x.reset_index(drop=True, inplace=True)
bottom10_y.reset_index(drop=True, inplace=True)

top10_x.columns = ['Top Nuptiality Region', 'Excessive Marriages per 100k']
top10_y.columns = ['Top Casualties Region', 'Crowdsourced Data']
bottom10_x.columns = ['Bottom Nuptiality Region', 'Excessive Marriages per 100k']
bottom10_y.columns = ['Bottom Casualties Region', 'Crowdsourced Data']

combined = pd.concat([top10_x, top10_y, bottom10_x, bottom10_y], axis=1)
combined


,Top Nuptiality Region,Excessive Marriages per 100k,Top Mortality Region,Crowdsourced Data,Bottom Nuptiality Region,Excessive Marriages per 100k,Bottom Mortality Region,Crowdsourced Data
0,Амурская область,826.227387,Республика Тыва,1188.992337,Чеченская Республика,-194.412186,Москва,35.090935
1,Сахалинская область,656.549063,Республика Бурятия,908.960522,Республика Дагестан,-158.233246,Санкт-Петербург,61.656782
2,Еврейская АО,641.219992,Республика Алтай,890.217974,Республика Ингушетия,-76.965366,Чеченская Республика,74.858351
3,Республика Бурятия,631.933737,Чукотский АО,695.206733,Северная Осетия,-61.928191,Республика Ингушетия,85.517073
4,Забайкальский край,610.334151,Сахалинская область,655.710201,Кабардино-Балкария,-33.479202,Кабардино-Балкария,101.699571
5,Приморский край,570.408257,Забайкальский край,596.446147,Чувашская Республика,55.497414,Московская область,122.114216
6,Республика Тыва,566.207472,Кировская область,574.928495,Астраханская область,63.262407,Тульская область,140.837797
7,Республика Алтай,533.093637,Ненецкий АО,554.850983,Удмуртская Республика,77.795176,Республика Дагестан,149.403490
8,Хабаровский край,526.467622,Республика Карелия,514.408192,Республика Мордовия,88.536207,Карачаево-Черкесия,151.858617
9,Республика Хакасия,518.869215,Республика Хакасия,513.122246,Тамбовская область,91.418303,Севастополь,154.926484


In [ ]:

fig = px.scatter(
    df_full,
    x='Excessive Marriages per100k',
    y='Total',
    hover_name='Region',
    title="Crowdsourced Casualties and Excessive Nuptiality",
    labels={"Excessive Marriages per100k": "Excessive Nuptiality", "Total": "Crowdsourced data"},
    width=800, height=700
)



fig.show()


## Excessive Mortality and Excessive Nuptiality

In [121]:
top10_x = df_full.nlargest(10, 'Excessive Marriages per100k')[['Region', 'Excessive Marriages per100k']]
top10_y = df_full.nlargest(10, 'Excess total')[['Region', 'Total']]
bottom10_x = df_full.nsmallest(10, 'Excessive Marriages per100k')[['Region', 'Excessive Marriages per100k']]
bottom10_y = df_full.nsmallest(10, 'Excess total')[['Region', 'Total']]

top10_x.reset_index(drop=True, inplace=True)
top10_y.reset_index(drop=True, inplace=True)
bottom10_x.reset_index(drop=True, inplace=True)
bottom10_y.reset_index(drop=True, inplace=True)

top10_x.columns = ['Top Nuptiality Region', 'Excessive Marriages per 100k']
top10_y.columns = ['Top Mortality Region', 'Excessive Male Mortality per100k']
bottom10_x.columns = ['Bottom Nuptiality Region', 'Excessive Marriages per 100k']
bottom10_y.columns = ['Bottom Mortality Region', 'Excessive Male Mortality per 100k']

combined = pd.concat([top10_x, top10_y, bottom10_x, bottom10_y], axis=1)
combined


,Top Nuptiality Region,Excessive Marriages per 100k,Top Mortality Region,Excessive Male Mortality per100k,Bottom Nuptiality Region,Excessive Marriages per 100k,Bottom Mortality Region,Excessive Male Mortality per 100k
0,Амурская область,826.227387,Амурская область,221.591878,Чеченская Республика,-194.412186,Республика Калмыкия,370.365607
1,Сахалинская область,656.549063,Удмуртская Республика,447.941649,Республика Дагестан,-158.233246,Республика Ингушетия,85.517073
2,Еврейская АО,641.219992,Севастополь,154.926484,Республика Ингушетия,-76.965366,Камчатский край,452.800577
3,Республика Бурятия,631.933737,Курская область,254.342370,Северная Осетия,-61.928191,Республика Алтай,890.217974
4,Забайкальский край,610.334151,Ненецкий АО,554.850983,Кабардино-Балкария,-33.479202,Калининградская область,310.314064
5,Приморский край,570.408257,Ульяновская область,331.820447,Чувашская Республика,55.497414,Карачаево-Черкесия,151.858617
6,Республика Тыва,566.207472,Приморский край,337.210991,Астраханская область,63.262407,Мурманская область,242.637135
7,Республика Алтай,533.093637,Сахалинская область,655.710201,Удмуртская Республика,77.795176,Вологодская область,351.687033
8,Хабаровский край,526.467622,Костромская область,354.584689,Республика Мордовия,88.536207,Рязанская область,174.913111
9,Республика Хакасия,518.869215,Республика Мордовия,219.174759,Тамбовская область,91.418303,Чувашская Республика,284.364441


In [ ]:

fig = px.scatter(
    df_full,
    x='Excessive Marriages per100k',
    y='Excess total',
    hover_name='Region',
    title="Excessive Mortality and Excessive Nuptiality",
    labels={"Excessive Marriages per100k": "Excessive Nuptiality", "Excess total": "Excessive mortality"},
    width=800, height=700
)



fig.show()


# Only confirmed drafted Deaths

In [ ]:

fig = px.scatter(
    df_full,
    x='Excessive Marriages per100k',
    y='Drafted',
    hover_name='Region',
    title="Crowdsourced Casualties (Drafted only) and Excessive Nuptiality",
    labels={"Excessive Marriages per100k": "Excessive Nuptiality", "Drafted": "Crowdsourced data"},
    width=800, height=700
)



fig.show()


In [ ]:

fig = px.scatter(
    df_full,
    x='Excessive Marriages per100k',
    y='Volunteers',
    hover_name='Region',
    title="Crowdsourced Casualties (Volunteers only) and Excessive Nuptiality",
    labels={"Excessive Marriages per100k": "Excessive Nuptiality", "Volunteers": "Crowdsourced data"},
    width=800, height=700
)



fig.show()


# Deposits

In [113]:
dep = pd.read_excel("C:/Users/aagal/projects/excess-mortality-war/deposits.xlsx", header = 1)[:95]

dep["pmgrowth"] = (dep["01.10.2023"].astype(int)/dep["01.10.2022"].astype(int)-1)*100
dep = dep.rename(columns={dep.columns[0] : "Region"})

dep = dep[["Region", "pmgrowth"]]
dep

,Region,pmgrowth
0,РОССИЙСКАЯ ФЕДЕРАЦИЯ,24.559147
1,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,25.464371
2,Белгородская область,17.172315
3,Брянская область,22.611742
4,Владимирская область,20.380628
...,...,...
90,Хабаровский край,22.032001
91,Амурская область,32.131201
92,Магаданская область,2.857883
93,Сахалинская область,20.944461


In [114]:
region_name_mapping = {
    'Архангельская область без данных по Ненецкому автономному округу': 'Архангельская область без АО',
    'в том числе Ненецкий автономный округ': 'Ненецкий АО',
    'Кемеровская область - Кузбасс': 'Кемеровская область',
    'Город Москва столица Российской Федерации город федерального значения': 'Москва',
    'Город Санкт-Петербург город федерального значения': 'Санкт-Петербург',
    'Город федерального значения Севастополь': 'Севастополь',
    'Еврейская автономная область': 'Еврейская АО',
    'Кабардино-Балкарская Республика': 'Кабардино-Балкария',
    'Карачаево-Черкесская Республика': 'Карачаево-Черкесия',
    'Республика Адыгея (Адыгея)': 'Республика Адыгея',
    'Республика Саха (Якутия)': 'Якутия',
    'Республика Северная Осетия - Алания': 'Северная Осетия',
    'Республика Татарстан (Татарстан)': 'Республика Татарстан',
    'Чувашская Республика - Чувашия': 'Чувашская Республика',
    'Тюменская область без данных по Ханты-Мансийскому автономному округу - Югре и Ямало-Ненецкому автономному округу': 'Тюменская область без АО',
    'в том числе Ханты-Мансийский автономный округ - Югра': 'Ханты-Мансийский АО',
    'в том числе Ямало-Ненецкий автономный округ': 'Ямало-Hенецкий АО',
    'Чукотский автономный округ': 'Чукотский АО',
    'г. Москва' : 'Москва',
    'г. Санкт-Петербург' : 'Санкт-Петербург',
    'г. Севастополь' : 'Севастополь'
}

dep['Region'] = dep['Region'].replace(region_name_mapping)



In [115]:
df_full = df_full.merge(dep, on = "Region", how = "left")

## Deposits and Nuptiality

In [125]:
top10_x = df_full.nlargest(10, 'Excessive Marriages per100k')[['Region', 'Excessive Marriages per100k']]
top10_y = df_full.nlargest(10, 'pmgrowth')[['Region', 'pmgrowth']]
bottom10_x = df_full.nsmallest(10, 'Excessive Marriages per100k')[['Region', 'Excessive Marriages per100k']]
bottom10_y = df_full.nsmallest(10, 'pmgrowth')[['Region', 'pmgrowth']]

top10_x.reset_index(drop=True, inplace=True)
top10_y.reset_index(drop=True, inplace=True)
bottom10_x.reset_index(drop=True, inplace=True)
bottom10_y.reset_index(drop=True, inplace=True)

top10_x.columns = ['Top Nuptiality Region', 'Excessive Marriages per 100k']
top10_y.columns = ['Top Deposits growth', 'Deposits yoy growth in October 2023']
bottom10_x.columns = ['Bottom Nuptiality Region', 'Excessive Marriages per 100k']
bottom10_y.columns = ['Bottom Deposits growth', 'Deposits yoy growth in October 2023']

combined = pd.concat([top10_x, top10_y, bottom10_x, bottom10_y], axis=1)
combined


,Top Nuptiality Region,Excessive Marriages per 100k,Top Deposits growth,Deposits yoy growth in October 2023,Bottom Nuptiality Region,Excessive Marriages per 100k,Bottom Deposits growth,Deposits yoy growth in October 2023
0,Амурская область,826.227387,Республика Тыва,51.782464,Чеченская Республика,-194.412186,Магаданская область,2.857883
1,Сахалинская область,656.549063,Республика Бурятия,37.583165,Республика Дагестан,-158.233246,Камчатский край,15.529303
2,Еврейская АО,641.219992,Республика Алтай,37.358044,Республика Ингушетия,-76.965366,Орловская область,16.760955
3,Республика Бурятия,631.933737,Краснодарский край,34.997547,Северная Осетия,-61.928191,Архангельская область без АО,16.962484
4,Забайкальский край,610.334151,Севастополь,34.443390,Кабардино-Балкария,-33.479202,Тамбовская область,17.043422
5,Приморский край,570.408257,Чеченская Республика,32.977254,Чувашская Республика,55.497414,Белгородская область,17.172315
6,Республика Тыва,566.207472,Ненецкий АО,32.810168,Астраханская область,63.262407,Нижегородская область,17.320204
7,Республика Алтай,533.093637,Амурская область,32.131201,Удмуртская Республика,77.795176,Калужская область,17.687535
8,Хабаровский край,526.467622,Республика Адыгея,30.956516,Республика Мордовия,88.536207,Республика Коми,17.854898
9,Республика Хакасия,518.869215,Республика Хакасия,29.854581,Тамбовская область,91.418303,Республика Карелия,17.887302


In [ ]:

fig = px.scatter(
    df_full,
    y='Excessive Marriages per100k',
    x='pmgrowth',
    hover_name='Region',
    title="Deposits growth in October 2023 (yoy) and Excessive Nuptiality",
    labels={"Excessive Marriages per100k": "Excessive Nuptiality", "pmgrowth": "Deposits growth"},
    width=800, height=700
)


fig.show()


## Deposits and Casualties (crowdsourced)

In [127]:
top10_x = df_full.nlargest(10, 'Total')[['Region', 'Total']]
top10_y = df_full.nlargest(10, 'pmgrowth')[['Region', 'pmgrowth']]
bottom10_x = df_full.nsmallest(10, 'Total')[['Region', 'Total']]
bottom10_y = df_full.nsmallest(10, 'pmgrowth')[['Region', 'pmgrowth']]

top10_x.reset_index(drop=True, inplace=True)
top10_y.reset_index(drop=True, inplace=True)
bottom10_x.reset_index(drop=True, inplace=True)
bottom10_y.reset_index(drop=True, inplace=True)

top10_x.columns = ['Top Casualties Region', 'Crowdsourced data']
top10_y.columns = ['Top Deposits growth', 'Deposits yoy growth in October 2023']
bottom10_x.columns = ['Bottom Casualties Region', 'Crowdsourced data']
bottom10_y.columns = ['Bottom Deposits growth', 'Deposits yoy growth in October 2023']

combined = pd.concat([top10_x, top10_y, bottom10_x, bottom10_y], axis=1)
combined


,Top Casualties Region,Crowdsourced data,Top Deposits growth,Deposits yoy growth in October 2023,Bottom Casualties Region,Crowdsourced data,Bottom Deposits growth,Deposits yoy growth in October 2023
0,Республика Тыва,1188.992337,Республика Тыва,51.782464,Москва,35.090935,Магаданская область,2.857883
1,Республика Бурятия,908.960522,Республика Бурятия,37.583165,Санкт-Петербург,61.656782,Камчатский край,15.529303
2,Республика Алтай,890.217974,Республика Алтай,37.358044,Чеченская Республика,74.858351,Орловская область,16.760955
3,Чукотский АО,695.206733,Краснодарский край,34.997547,Республика Ингушетия,85.517073,Архангельская область без АО,16.962484
4,Сахалинская область,655.710201,Севастополь,34.443390,Кабардино-Балкария,101.699571,Тамбовская область,17.043422
5,Забайкальский край,596.446147,Чеченская Республика,32.977254,Московская область,122.114216,Белгородская область,17.172315
6,Кировская область,574.928495,Ненецкий АО,32.810168,Тульская область,140.837797,Нижегородская область,17.320204
7,Ненецкий АО,554.850983,Амурская область,32.131201,Республика Дагестан,149.403490,Калужская область,17.687535
8,Республика Карелия,514.408192,Республика Адыгея,30.956516,Карачаево-Черкесия,151.858617,Республика Коми,17.854898
9,Республика Хакасия,513.122246,Республика Хакасия,29.854581,Севастополь,154.926484,Республика Карелия,17.887302


In [129]:

fig = px.scatter(
    df_full,
    y='Total',
    x='pmgrowth',
    hover_name='Region',
    title="Deposits growth in October 2023 (yoy) and Crowdsourced Casualties",
    labels={"pmgrowth": "Deposits Growth", "Total": "Crowdsourced Casualties"},
    width=800, height=700
)


fig.show()
